# Код Хаффмана

In [17]:
with open('file.txt', encoding='utf-8', mode='r') as file:
    text = file.read()

text

'Древность\n\nУстная передача — самый древний способ передачи знаний в истории человечества. После изобретения древними цивилизациями систем записи люди начали использовать для письма почти всё, на чём можно писать — глиняные таблички, кору дерева, листы металла и т. п.\nТаблички\nШумерская глиняная табличка с клинописью\n\nТабличку можно определить как физически прочный, надёжный носитель письменной информации, относительно удобный в повседневном использовании и транспортировке. Пишущим средством в этом случае, как правило, выступало стило. Можно выделить два основных типа табличек: глиняные (например, у населения долины между Тигром и Евфратом), которые часто использовались для письма клинописью[4], и восковые. Последние представляли собой дощечки, покрытые слоем воска, в то время как глиняные полностью состояли из глины и после нанесения надписей часто обжигались для придания им дополнительной прочности. После этой процедуры, соответственно, изменить текст было уже невозможно; напро

In [18]:
from collections import Counter


class NodeTree(object):
    def __init__(self, left=None, right=None):
        self.left = left
        self.right = right

    def __str__(self):
        return self.left, self.right

    def children(self):
        return self.left, self.right


def huffman_code_tree(node_, bin_string=''):
    if type(node_) is str:
        return {node_: bin_string}

    l, r = node_.children()
    d = dict()
    d.update(huffman_code_tree(l, bin_string + '0'))
    d.update(huffman_code_tree(r, bin_string + '1'))
    return d


def make_tree(nodes):
    while len(nodes) > 1:
        key1, c1 = nodes[-1]
        key2, c2 = nodes[-2]
        nodes = nodes[:-2]
        node_ = NodeTree(key1, key2)
        nodes.append((node_, c1 + c2))
        nodes = sorted(nodes, key=lambda x: x[1], reverse=True)

    return nodes[0][0]

In [19]:
import pandas as pd

freq = sorted(
    dict(Counter(text)).items(),
    key=lambda kv: kv[1],
    reverse=True,
)

node = make_tree(freq)
encoding = huffman_code_tree(
    make_tree(freq)
)

letter_list = list(encoding.keys())
code_words = list(encoding.values())

pd.DataFrame({
    'Символ': letter_list,
    'Кодовое слово': code_words,
})

,Символ,Кодовое слово
0,с,0000
1,",",000100
2,ь,000101
3,м,00011
4,т,0010
...,...,...
109,a,1111111110101
110,N,1111111110110
111,D,1111111110111
112,«,11111111110


## Кодирование текста

In [20]:
coder = {
    c: code_words[i]
    for i, c in enumerate(letter_list)
}

encoded_message = ''.join(
    coder[c]
    for c in text
)

In [21]:
plain_len = len(''.join(format(ord(x), 'b') for x in text))
encoded_len = len(encoded_message)

print(f'Эффективность сжатия: {1 - (encoded_len / plain_len):.10f}')

Эффективность сжатия: 0.5192387291
